In [5]:
from omegaconf import OmegaConf
import numpy as np
import os
import re
import os.path as osp
import torch
import pytorch_lightning as pl
from tqdm import tqdm
from omegaconf import OmegaConf
import wandb
from pytorch_lightning.loggers import WandbLogger
from glob import glob
import time

import model_factory
from graph_data_module import GraphDataModule
from train import Runner
from datasets_torch_geometric.dataset_factory import create_dataset
from torch_geometric.loader import DataLoader
import matplotlib.pyplot as plt
import torchvision
import torchmetrics
from sklearn.metrics import ConfusionMatrixDisplay

In [6]:
entity = "haraghi"
project = "sweep EST (FAN1VS3) (multi val test num 20)"

In [8]:
run_ids = ['bnq6i70m']

run_id = run_ids[0]
artifact_dirs = []
for run_id in run_ids:
    glob_results = glob(osp.join(run_id, "checkpoints","*"))
    assert len(glob_results) == 1
    artifact_dirs.append(glob_results[0])

In [9]:
api = wandb.Api()

In [10]:
cfg_bare = OmegaConf.load("config_bare.yaml")
configs = [api.run(osp.join(entity, project, run_id)).config for run_id in run_ids]
cfgs = [OmegaConf.create(config) for config in configs]
cfg_files = []
for cfg in cfgs:
    if "cfg_path" in cfg.keys():
        print(cfg.cfg_path)
        cfg_files.append(OmegaConf.merge(cfg_bare,OmegaConf.load(cfg.cfg_path)))
    else:
        cfg_files.append(cfg)
            

cfgs = [OmegaConf.merge(cfg_file, cfg) for cfg_file, cfg in zip(cfg_files, cfgs)]
# print(OmegaConf.to_yaml(cfgs[0]))

cfg_folder/EST_FAN1VS3_1024_wandb_sweep.yaml


In [11]:
def recursive_dict_compare(all_cfg, other_cfg):
    """
    Recursively compare two dictionaries and return their differences.
    """

    
    # Initialize the result dictionary
    diff = {}

    # Check for keys in dict1 that are not in dict2
    for key in other_cfg:
        if key not in all_cfg:
            diff[key] = other_cfg[key]
        else:
            # If the values are dictionaries, recursively compare them
            if isinstance(all_cfg[key], dict) and isinstance(other_cfg[key], dict):
                nested_diff = recursive_dict_compare(all_cfg[key], other_cfg[key])
                if nested_diff:
                    diff[key] = nested_diff
            # Otherwise, compare the values directly
            elif all_cfg[key] != other_cfg[key]:
                if not(key == "num_classes" and other_cfg[key] is None and all_cfg[key] is not None):
                    diff[key] = other_cfg[key]
                    

    return diff


In [12]:
print([recursive_dict_compare(OmegaConf.to_object(cfg),OmegaConf.to_object(cfg_file)) for cfg, cfg_file in zip(cfgs, cfg_files)])

[{'seed': 0, 'optimize': {'patience': 10}, 'train': {'batch_size': 8}, 'wandb': {'entity': '', 'project': ''}}]


In [13]:
# Seed everything. Note that this does not make training entirely
# deterministic.
for cfg in cfgs:
    pl.seed_everything(cfg.seed, workers=True)

for cfg in cfgs[1:]:
    compare_dict = recursive_dict_compare(OmegaConf.to_object(cfgs[0].dataset),OmegaConf.to_object(cfg.dataset))
    if len(compare_dict)!=0:
        if not (len(compare_dict) == 1 and 'num_workers' in compare_dict.keys()):
            print(compare_dict)
            print(cfg.dataset)
            print(cfgs[0].dataset)
            # raise Exception("Datasets are not the same")
# Create datasets using factory pattern


gdm = GraphDataModule(cfgs[0])
for cfg in cfgs:
    cfg.dataset.num_classes = gdm.num_classes

Global seed set to 7


In [14]:
trainloader = gdm.train_dataloader()
torch.save(trainloader, "trainloader.pt")
testloader = gdm.test_dataloader()[0]
torch.save(testloader, "testloader.pt")

In [15]:
models = [model_factory.factory(cfg) for cfg in cfgs]

# Tie it all together with PyTorch Lightning: Runner contains the model,
# optimizer, loss function and metrics; Trainer executes the
# training/validation loops and model checkpointing.
 
runners = [Runner.load_from_checkpoint(artifact_dir, cfg=cfg, model=model) for artifact_dir, cfg, model in zip(artifact_dirs, cfgs, models)]

gdms = [GraphDataModule(cfg) for cfg in cfgs]
dss = []
for cfg,gdm in zip(cfgs, gdms):
    cfg.dataset.num_classes = gdm.num_classes

    dss.append(create_dataset(
        dataset_path = gdm.dataset_path,
        dataset_name  = gdm.dataset_name,
        dataset_type = 'test',
        transform = gdm.transform_dict['test'],
        num_workers=gdm.num_workers
    ))

In [19]:
torch.save(runners[0].model, "model.pt")

In [29]:
trainer = pl.Trainer(
    enable_progress_bar=True,
    # Use DDP training by default, even for CPU training
    # strategy="ddp_notebook",
    devices=torch.cuda.device_count(),
    accelerator="auto"
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


What is the difference between runner.parameters() vs runner.stat_dict

In [30]:
wtf = torch.load(artifact_dirs[0])  

In [55]:
stat_dict_set = set()
for k,v in wtf['state_dict'].items():
    stat_dict_set.add(k)

In [54]:
param_set = set()
for nmae in runners[0].named_parameters():
    param_set.add(nmae[0])

In [59]:
difference = stat_dict_set - param_set
for d in difference:
    print(d)

model.classifier.layer4.2.bn1.running_var
model.classifier.layer1.2.bn2.num_batches_tracked
model.classifier.layer2.1.bn1.running_mean
model.classifier.layer2.0.bn2.num_batches_tracked
model.classifier.layer4.0.downsample.1.running_var
model.classifier.layer3.0.bn1.num_batches_tracked
model.classifier.layer2.0.downsample.1.running_var
model.classifier.layer3.4.bn1.num_batches_tracked
model.classifier.layer3.3.bn2.running_mean
model.classifier.layer2.3.bn2.running_mean
model.classifier.layer4.1.bn1.running_var
model.classifier.layer2.0.downsample.1.running_mean
model.classifier.layer4.0.bn1.running_var
model.classifier.layer2.0.downsample.1.num_batches_tracked
model.classifier.layer3.0.downsample.1.running_var
model.classifier.layer3.0.bn2.running_mean
model.classifier.layer4.0.bn2.running_var
model.classifier.layer3.2.bn2.running_mean
model.classifier.layer2.2.bn1.num_batches_tracked
model.classifier.layer3.2.bn1.num_batches_tracked
model.classifier.layer4.2.bn2.running_mean
model.clas

In [84]:
num_test = 10


for runner in runners:
    print(runner.cfg.model)
    acc_list = []
    duration_list = []
    for i in range(num_test):
        start_time = time.time()
        test_results = trainer.test(runner, datamodule=gdm, verbose=False)
        end_time = time.time()
        duration_list.append(end_time-start_time)
        acc_list.append(test_results[0]['test/acc_mean'])
    print(f"acc: {np.mean(acc_list)} +- {np.std(acc_list)}")
    print(f"duration: {np.mean(duration_list)} +- {np.std(duration_list)}")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


{'name': 'EST', 'k': None, 'aggr': '', 'num_bins': 9, 'cnn_type': 'resnet34', 'resnet_crop_dimension': [224, 224], 'est_mlp_layers': [1, 30, 30, 1], 'est_activation': 'nn.LeakyReLU(negative_slope=0.1)', 'resnet_pretrained': True}
Testing DataLoader 19: 100%|██████████| 3/3 [00:00<00:00,  7.90it/s]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 19: 100%|██████████| 3/3 [00:00<00:00,  7.99it/s]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f9866d43880>
Traceback (most recent call last):
  File "/home/hesam/miniconda3/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/home/hesam/miniconda3/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1443, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/home/hesam/miniconda3/lib/python3.10/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/home/hesam/miniconda3/lib/python3.10/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
  File "/home/hesam/miniconda3/lib/python3.10/multiprocessing/connection.py", line 936, in wait
    ready = selector.select(timeout)
  File "/home/hesam/miniconda3/lib/python3.10/selectors.py", line 416, in select
    fd_event_list = self._selec

Testing DataLoader 8: 100%|██████████| 3/3 [00:00<00:00,  8.35it/s]

In [74]:
gdm.test_dataloader()[0].dataset

FAN1VS3(78)

In [ ]:
# assuming you have a list of accuracy values called `acc_list`
plt.hist(acc_list)
plt.xlabel('Accuracy')
plt.ylabel('Frequency')
plt.title('Histogram of Accuracy')
plt.show()
print(np.mean(acc_list))
print(np.std(acc_list))

In [ ]:
dl = DataLoader(
            create_dataset(
                dataset_path = gdm.dataset_path,
                dataset_name  = gdm.dataset_name,
                dataset_type = 'test',
                transform = gdm.transform_dict['test'],
                num_workers=gdm.num_workers
            ),
            batch_size=cfg.train.batch_size,
            shuffle=False,
            num_workers=gdm.num_workers,
        )
ds = create_dataset(
        dataset_path = gdm.dataset_path,
        dataset_name  = gdm.dataset_name,
        dataset_type = 'test',
        transform = gdm.transform_dict['test'],
        num_workers=gdm.num_workers
    )

In [ ]:
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = runners[0].model.to(device)
model.eval()

correct = 0
total = 0

y=torch.tensor([],device=device)
y_hat=torch.tensor([],device=device)
preds = []
targets = []
files = []
with torch.no_grad():
    for data in tqdm(gdm.test_dataloader()):
        files.extend(data.file_id)
        targets.append(data.y)
        data = data.to(device)
        y = torch.cat((y,data.y))
        out = model(data)
        preds.append(out.clone().detach().cpu())
        label = torch.argmax(out, dim=1) 
        y_hat = torch.cat((y_hat,label))
        correct += torch.sum(label == data.y)
        total += data.y.shape[0]
  
y = y.clone().detach().cpu().numpy()
y_hat = y_hat.clone().detach().cpu().numpy() 
preds_ = torch.cat(preds,dim=0) #.permute(0,2,1)
targets_ = torch.cat(targets,dim=0)
metrics = torchmetrics.classification.Accuracy(num_classes=runner.cfg.dataset.num_classes, task="multiclass", top_k=1) 

acc = metrics(preds_, targets_).detach().cpu().numpy()
print(acc)
# return confusion_matrix_computed, y, y_hat, files

In [ ]:
class2ind = {c:[] for c in ds.categories}
for i,d in enumerate(ds):
    
    class2ind[d.label[0]].append(i)

In [ ]:
name2ind = {d.file_id: i  for i, d in enumerate(ds)}

In [ ]:
idx = torch.randint(0, len(ds),(1,))
# idx = [6]
# idx = (name2ind['l_0011.mat'],)
correct = 0
total = 0

y1_hat = []
model.eval()

with torch.no_grad():
    for ii in tqdm(range(1000)):
        
        data = ds[idx][0]
        data.batch = torch.zeros(data.num_nodes, dtype=torch.long)
        data = data.to(device)
        
        out = model(data)
        label = torch.argmax(out, dim=1)
        y1_hat.append(label.cpu().item())
        correct += torch.sum(label == data.y)
        total += data.y.shape[0]
    # loss = runner.loss_fn(y_hat, y)
print(data.file_id)  
print(data.label[0])
print(correct/total)
print([ds.categories[int(c)] for c in np.unique(np.array(y1_hat))])